In [6]:
import ccxt
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone

# --- 常數定義 ---
DAYS_PER_YEAR = 365.25
SECONDS_PER_DAY = 24 * 3600

# --- CCXT K 線資料抓取 ---
def get_kline_data(exchange_id: str, symbol: str, timeframe: str = "1h", limit: int = 720) -> pd.DataFrame:
    """
    使用 CCXT 從指定交易所取得歷史 K 線資料
    """
    try:
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({
            'enableRateLimit': True,
            'options': {'defaultType': 'spot'} # 預設使用現貨，可根據需要調整
        })
        
        # 檢查並載入市場數據以確保符號正確
        exchange.load_markets()
        
        if symbol not in exchange.markets:
            # 嘗試自動修正常見格式差異 (例如 gate 的 BTC_USDT -> BTC/USDT)
            if '_' in symbol and '/' not in symbol:
                symbol = symbol.replace('_', '/')
            
            if symbol not in exchange.markets:
                raise ValueError(f"交易所 {exchange_id} 不支持交易對 {symbol}")

        # 獲取 K 線數據: [timestamp, open, high, low, close, volume]
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        
        df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
        
        # 依時間排序
        df = df.sort_values("timestamp").reset_index(drop=True)
        
        return df[["timestamp", "open", "high", "low", "close"]]
        
    except Exception as e:
        print(f"獲取 K 線數據失敗: {e}")
        return pd.DataFrame()


# --- 核心策略計算 ---
def calculate_optimal_market_making_params(
    asset: str,
    mid_price: float,
    daily_volatility_pct: float,
    target_order_fill_prob: float,
    order_refresh_time_sec: int,
    stop_loss_risk_prob: float,
    max_holding_time_days: float = 30.0,
    profit_factor: float = 2.0
) -> dict:
    """
    根據 GBM 波動率模型，計算最優造市參數
    """
    daily_volatility = daily_volatility_pct / 100.0
    annual_volatility = daily_volatility * np.sqrt(DAYS_PER_YEAR)
    dt_order = order_refresh_time_sec / (DAYS_PER_YEAR * SECONDS_PER_DAY)
    dt_loss = max_holding_time_days / DAYS_PER_YEAR

    # 基礎掛單價差
    p_half_order = target_order_fill_prob / 2.0
    Z_order = norm.ppf(p_half_order)
    base_spread_pct = (annual_volatility * np.sqrt(dt_order) * np.abs(Z_order)) * 100

    # 止盈與止損
    profit_taking_spread_pct = base_spread_pct * profit_factor
    p_half_loss = stop_loss_risk_prob / 2.0
    Z_loss = norm.ppf(p_half_loss)
    stop_loss_spread_pct = (annual_volatility * np.sqrt(dt_loss) * np.abs(Z_loss)) * 100

    return {
        "asset": asset,
        "current_mid_price": mid_price,
        "order_refresh_time_sec": order_refresh_time_sec,
        "bid_spread": round(base_spread_pct, 4),
        "ask_spread": round(base_spread_pct, 4),
        "long_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "short_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "stop_loss_spread": round(stop_loss_spread_pct, 4),
        "Z_score_order": round(np.abs(Z_order), 4),
        "Z_score_stop_loss": round(np.abs(Z_loss), 4)
    }


# --- 主流程：自動從交易所取得波動率並計算策略 ---
def calculate_strategy(exchange_id: str, symbol: str, timeframe: str):
    """
    從指定交易所取得歷史資料，自動估算波動率並計算造市策略參數
    """
    print(f"正在從 {exchange_id} 獲取 {symbol} 的 {timeframe} K線數據...")
    df = get_kline_data(exchange_id, symbol, timeframe=timeframe, limit=720)

    if df.empty:
        print("無法獲取數據，程序終止。")
        return

    # 計算對數報酬率
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    
    # 計算標準差 (波動率)
    period_vol = df["log_return"].std()

    # 根據 timeframe 換算成日化波動率
    # 假設 timeframe 格式為 '1m', '5m', '1h', '1d' 等
    intervals_per_day = 24 # Default 1h
    if timeframe == '1m':
        intervals_per_day = 1440
    elif timeframe == '5m':
        intervals_per_day = 288
    elif timeframe == '15m':
        intervals_per_day = 96
    elif timeframe == '30m':
        intervals_per_day = 48
    elif timeframe == '1h':
        intervals_per_day = 24
    elif timeframe == '4h':
        intervals_per_day = 6
    elif timeframe == '1d':
        intervals_per_day = 1
    
    daily_vol = period_vol * np.sqrt(intervals_per_day)
    print(f"📊 {symbol} 日化波動率估計值: {daily_vol*100:.2f}%")

    params = {
        "asset": symbol,
        "mid_price": df["close"].iloc[-1],
        "daily_volatility_pct": daily_vol * 100,
        "target_order_fill_prob": 0.3,
        "order_refresh_time_sec": 30,
        "stop_loss_risk_prob": 0.2,
        "max_holding_time_days": 1,
        "profit_factor": 1
    }

    result = calculate_optimal_market_making_params(**params)

    print("\n--- 🔬 最優造市參數計算結果 ---")
    for key, value in result.items():
        if "_spread" in key or "time_sec" in key:
            unit = "%" if "spread" in key else "秒"
            print(f"{key:<30}: {value:>.4f} {unit}")
        elif key == "current_mid_price":
            print(f"{key:<30}: {value:>.4f} USDT")
        elif key == "asset":
            print(f"{key:<30}: {value}")

    return result


# --- 主執行 ---
if __name__ == "__main__":
    # 傳入參數
    exchange_id = 'bybit' # 支援主流交易所 可參照以下輸出
    symbol = 'BTC/USDT' # 需加上/USDT或/USDC等
    timeframe = '1h' # 格式為 '1m', '5m', '1h', '1d' 等
    calculate_strategy(exchange_id, symbol, timeframe)


正在從 bybit 獲取 BTC/USDT 的 1h K線數據...
📊 BTC/USDT 日化波動率估計值: 2.74%

--- 🔬 最優造市參數計算結果 ---
asset                         : BTC/USDT
current_mid_price             : 89029.4000 USDT
order_refresh_time_sec        : 30.0000 秒
bid_spread                    : 0.0528 %
ask_spread                    : 0.0528 %
long_profit_taking_spread     : 0.0528 %
short_profit_taking_spread    : 0.0528 %
stop_loss_spread              : 3.5057 %


### ccxt支援的交易所列表

In [7]:
from pprint import pprint
import ccxt  # noqa: E402

print('CCXT Version:', ccxt.__version__)

for exchange_id in ccxt.exchanges:
    try:
        exchange = getattr(ccxt, exchange_id)()
        print(exchange_id)
        # do what you want with this exchange
        # pprint(dir(exchange))
    except Exception as e:
        print(e)

CCXT Version: 4.5.2
alpaca
apex
ascendex
bequant
bigone
binance
binancecoinm
binanceus
binanceusdm
bingx
bit2c
bitbank
bitbns
bitfinex
bitflyer
bitget
bithumb
bitmart
bitmex
bitopro
bitrue
bitso
bitstamp
bitteam
bittrade
bitvavo
blockchaincom
blofin
btcalpha
btcbox
btcmarkets
btcturk
bybit
cex
coinbase
coinbaseadvanced
coinbaseexchange
coinbaseinternational
coincatch
coincheck
coinex
coinmate
coinmetro
coinone
coinsph
coinspot
cryptocom
cryptomus
defx
delta
deribit
derive
digifinex
exmo
fmfwio
foxbit
gate
gateio
gemini
hashkey
hibachi
hitbtc
hollaex
htx
huobi
hyperliquid
independentreserve
indodax
kraken
krakenfutures
kucoin
kucoinfutures
latoken
lbank
luno
mercado
mexc
modetrade
myokx
ndax
novadax
oceanex
okcoin
okx
okxus
onetrading
oxfun
p2b
paradex
paymium
phemex
poloniex
probit
timex
tokocrypto
tradeogre
upbit
wavesexchange
whitebit
woo
woofipro
xt
yobit
zaif
zonda
